In [1]:
# 모듈 임포트
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.contrib import layers

In [2]:
# 트레이닝 데이터 로드
x_data = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_activity.csv').sort_values(by='acc_id', ascending=True)
y_data = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\train\\train_label.csv').sort_values(by='acc_id', ascending=True)

In [3]:
# 트레이닝데이터 아이디별로 그룹화 및 평균
# 데이터의 시계열 특성이 사라지지만, 빨리 단순 회귀 모형을 볼 수 있음
x_data = x_data.groupby(by='acc_id').mean().values
y_data = y_data.label.values

In [4]:
#트레이닝셋, 발리데이션셋 구분
x_train = x_data[0:int(0.7 * len(x_data)),:]
y_train = y_data[0:int(0.7 * len(x_data))]

x_cv = x_data[int(0.7 * len(x_data)):,:]
y_cv = y_data[int(0.7 * len(x_data)):]

In [5]:
#한국어로 된 레이블을 숫자로 인코딩
idx2label = list(set(y_train))
label2idx = {c:i for i,c in enumerate(idx2label)}

#new y_train & y_cv
y_train = [label2idx[v] for v in y_train]
y_cv = [label2idx[v] for v in y_cv]

### CONSTANT
---

In [90]:
# 레이블 개수
num_feature = len(x_train[0])
learning_rate = 0.01
training_epoch = 100
batch_size = 100
label = len(idx2label)

### TENSORFLOW
---

In [127]:
X = tf.placeholder(tf.float32, [None, num_feature])
Y = tf.placeholder(tf.int32, [None])
keep_prob = tf.placeholder(tf.float32)

In [128]:
#Y_one_hot
Y_one_hot = tf.one_hot(Y, label)

In [129]:
#Layer1
with tf.name_scope('layer1') as scope:
    w1 = tf.get_variable("weight1", [num_feature, 64], tf.float32, layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([64]), name = "bias1")
    
    layer1 = tf.nn.relu(tf.matmul(X,w1) + b1)
    layer1 = tf.nn.dropout(layer1, keep_prob)

In [130]:
#Layer2
with tf.name_scope('layer2') as scope:
    w2 = tf.get_variable("weight2", [64, 32], tf.float32, layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([32]), name = "bias2")
    
    layer2 = tf.nn.relu(tf.matmul(layer1,w2) + b2)
    layer2 = tf.nn.dropout(layer2, keep_prob)

In [131]:
#Layer3
with tf.name_scope('layer3') as scope:
    w3 = tf.get_variable("weight3", [32, 16], tf.float32, layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([16]), name = "bias3")
    
    layer3 = tf.nn.relu(tf.matmul(layer2,w3) + b3)
    layer3 = tf.nn.dropout(layer3, keep_prob)

In [132]:
#Layer4
with tf.name_scope('layer4') as scope:
    w4 = tf.get_variable("weight4", [16, 8], tf.float32, layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([8]), name = "bias4")
    
    layer4 = tf.nn.relu(tf.matmul(layer3,w4) + b4)
    layer4 = tf.nn.dropout(layer4, keep_prob)

In [133]:
#hypothesis
with tf.name_scope('hypothesis') as scope:
    w5 = tf.get_variable("weight5", [8, label], tf.float32, layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([label]), name = "bias5")
    
    logits = tf.matmul(layer4, w5) + b5

In [134]:
#Cost Function - softmax cross entropy
with tf.name_scope("cost") as scope:
    cost = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y_one_hot)
    cost = tf.reduce_mean(cost)
    tf.summary.scalar("cost", cost)

In [135]:
#Train - Gradient Descent
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train = optimizer.minimize(cost)

In [136]:
#Prediction
prediction = tf.argmax(logits, -1)

In [137]:
#Accuracy
with tf.name_scope("accuracy") as scope:
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, tf.argmax(Y_one_hot,-1)), tf.float32))
    tf.summary.scalar("accuracy", accuracy)

### 저장!!!
---

In [138]:
saver = tf.train.Saver()

### 학습 !!!
---

In [139]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epoch):
        avg_cost = 0
        total_batch = int(len(x_train) / batch_size)
        
        for i in range(total_batch):
            batch_x = x_train[i*batch_size:(i+1)*batch_size]
            batch_y = y_train[i*batch_size:(i+1)*batch_size]
            
            for i in range(5):
                cost_val, _ = sess.run([cost, train], feed_dict={X:batch_x,Y:batch_y, keep_prob:0.7})
            avg_cost += cost_val/total_batch
        
        print("[epoch : {}]  cost {}".format(epoch, avg_cost))
    
    print("accuracy : {}".format(sess.run(accuracy, feed_dict={X:x_train, Y:y_train, keep_prob:1})))
    
    saver.save(sess, './02_simple_neural_network/model.ckpt')

[epoch : 0]  cost 1.3201327066762112
[epoch : 1]  cost 1.1805469095706944
[epoch : 2]  cost 1.128007692268918
[epoch : 3]  cost 1.0970134280409136
[epoch : 4]  cost 1.080416272963797
[epoch : 5]  cost 1.0654125440120705
[epoch : 6]  cost 1.0567918401105054
[epoch : 7]  cost 1.0447269667897905
[epoch : 8]  cost 1.0305627888441096
[epoch : 9]  cost 1.0241546753474642
[epoch : 10]  cost 1.0175259153332015
[epoch : 11]  cost 1.0168994973387033
[epoch : 12]  cost 1.0117021467855998
[epoch : 13]  cost 1.0051672978060582
[epoch : 14]  cost 1.0007153643880569
[epoch : 15]  cost 0.9966269832849494
[epoch : 16]  cost 0.992397767986571
[epoch : 17]  cost 0.9921547661508833
[epoch : 18]  cost 0.9886026209592821
[epoch : 19]  cost 0.9802987213645661
[epoch : 20]  cost 0.9859412051950182
[epoch : 21]  cost 0.978964198827743
[epoch : 22]  cost 0.9747651772839684
[epoch : 23]  cost 0.9714485952683858
[epoch : 24]  cost 0.9692179096596585
[epoch : 25]  cost 0.9709649121761316
[epoch : 26]  cost 0.96727

### 텐서플로 그래프 초기화
---

In [126]:
tf.reset_default_graph()

### 테스트 데이터
---

In [140]:
# 테스트 데이터 로드
x_test_temp = pd.read_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\test_activity.csv').sort_values(by='acc_id', ascending=True)

In [144]:
#label
label = []

In [141]:
#value
x_test = x_test_temp.groupby(by='acc_id').mean().values

In [145]:
with tf.Session() as sess:
    saver.restore(sess, './02_simple_neural_network/model.ckpt')
    
    for v in sess.run(prediction, feed_dict={X:x_test, keep_prob:1}) :
        label.append(idx2label[v])

INFO:tensorflow:Restoring parameters from ./02_simple_neural_network/model.ckpt


In [146]:
acc_id = x_test_temp.groupby(by='acc_id').mean().index

data = {'acc_id':acc_id, 'label':label}

In [147]:
result = pd.DataFrame(data)

In [150]:
result.to_csv('C:\\Users\\zeus_\\Desktop\\champion_data\\test\\result\\02_simple_neural_network.csv')